In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

# installeer lifelines
# from lifelines import KaplanMeierFitter
# from lifelines import CoxPHFitter

In [2]:
df = pd.read_csv("survival_data.csv")
df.loc[:, "censored"] = 1-df.broken
df.head()

,lifetime,broken,pressureInd,moistureInd,temperatureInd,team,provider,censored
0,56,0,92.178854,104.230204,96.517159,TeamA,Provider4,1
1,81,1,72.075938,103.065701,87.271062,TeamC,Provider4,0
2,60,0,96.272254,77.801376,112.196170,TeamA,Provider1,1
3,86,1,94.406461,108.493608,72.025374,TeamC,Provider2,0
4,34,0,97.752899,99.413492,103.756271,TeamB,Provider1,1


In [3]:
df_filter = df.drop(["pressureInd", "moistureInd", "temperatureInd", "team", "provider"], axis = 1)

df_sort = df_filter.sort_values(by = "lifetime")
df_sort = df_sort.reset_index(drop = True)
df_sort

,lifetime,broken,censored
0,1,0,1
1,1,0,1
2,1,0,1
3,1,0,1
4,1,0,1
...,...,...,...
995,93,1,0
996,93,1,0
997,93,1,0
998,93,1,0


In [4]:
df_group = df_sort.groupby(by = "lifetime").sum()
# first row calculations
df_group.loc[1, "nSamples"] = len(df_sort)
df_group.loc[1, "hf"] = df_group.broken[1]/df_group.nSamples[1]
df_group.loc[1, "chf"] = df_group.loc[1, "hf"]
df_group.loc[1, "survival"] = 1-df_group.loc[1, "chf"]
df_group

,broken,censored,nSamples,hf,chf,survival
lifetime,,,,,,
1,0,5,1000.0,0.0,0.0,1.0
2,0,11,NaN,NaN,NaN,NaN
3,0,3,NaN,NaN,NaN,NaN
4,0,8,NaN,NaN,NaN,NaN
5,0,10,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
88,42,4,NaN,NaN,NaN,NaN
89,1,0,NaN,NaN,NaN,NaN
90,0,1,NaN,NaN,NaN,NaN


In [5]:
prev_ind = 1
for ii in df_group.index:
    if ii == 1:
        continue
    
    df_group.loc[ii, "nSamples"] = df_group.loc[prev_ind, "nSamples"] - df_group.loc[prev_ind, "broken"] - df_group.loc[prev_ind, "censored"]
    df_group.loc[ii, "hf"] = df_group.broken[ii]/df_group.nSamples[ii]
    df_group.loc[ii, "chf"] = df_group.loc[ii, "hf"] + df_group.loc[prev_ind, "chf"]
    df_group.loc[ii, "survival"] = 1-df_group.loc[ii, "chf"]
    
    prev_ind = ii

df_group.loc[77:]

,broken,censored,nSamples,hf,chf,survival
lifetime,,,,,,
77,0,9,295.0,0.000000,0.336667,0.663333
78,0,5,286.0,0.000000,0.336667,0.663333
79,19,1,281.0,0.067616,0.404282,0.595718
80,66,5,261.0,0.252874,0.657156,0.342844
81,27,6,190.0,0.142105,0.799261,0.200739
82,6,1,157.0,0.038217,0.837478,0.162522
83,0,2,150.0,0.000000,0.837478,0.162522
84,0,2,148.0,0.000000,0.837478,0.162522
85,13,5,146.0,0.089041,0.926519,0.073481
